## Classify Text into Labels
As per this [tutorial](https://python.langchain.com/docs/tutorials/classification/)

Tagging means labeling a document with classes such as:
- Sentiment
- Language
- Style (formal, informal etc.)
- Covered topics
- Political tendency

Tagging has a few components:

- function: Like extraction, tagging uses functions to specify how the model should tag a document
- schema: defines how we want to tag the document

#### Quickstart
Let's see a very straightforward example of how we can use OpenAI tool calling for tagging in LangChain. We'll use the with_structured_output method supported by OpenAI models.

In [1]:
import os
from dotenv import load_dotenv

# Loading the .env file. Using full path to use it in the interactive shell
load_dotenv("C:\\Users\\MarcusChiri\\Dropbox\\Repositories\\Langchain\\.env")

# Loarding the environment variables
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

Let's specify a Pydantic model with a few properties and their expected type in our schema.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [4]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [5]:
inp = "Aqui tem um bando de loucos! Louco por ti Corinthians! Aqueles que acham que é pouco, eu vivo por ti Corinthians! Eu canto até ficar rouco! Eu canto pra te empurrar! Vamo vamo meu timão, vamo meu timão! Não para de lutar!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='passionate', aggressiveness=3, language='Portuguese')

In [7]:
inp = "Se o Corinthians não ganhar, o pau vai quebrar"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='negative', aggressiveness=8, language='Portuguese')

If we want dictionary output, we can just call .model_dump()

In [8]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response.model_dump()

{'sentiment': 'angry', 'aggressiveness': 8, 'language': 'Spanish'}

As we can see in the examples, it correctly interprets what we want.

The results vary so that we may get, for example, sentiments in different languages ('positive', 'enojado' etc.).

We will see how to control these results in the next section.

### Finer Control

Finer control
Careful schema definition gives us more control over the model's output.

Specifically, we can define:

Possible values for each property
Description to make sure that the model understands the property
Required properties to be returned
Let's redeclare our Pydantic model to control for each of the previously mentioned aspects using enums:

In [10]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [11]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification
)

Now the answers will be restricted in a way we expect!

In [12]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='happy', aggressiveness=1, language='spanish')

In [13]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='sad', aggressiveness=4, language='spanish')

## Going deeper
You can use the [metadata tagger](https://python.langchain.com/docs/integrations/document_transformers/openai_metadata_tagger) document transformer to extract metadata from a LangChain Document.
This covers the same basic functionality as the tagging chain, only applied to a LangChain Document.